In [1]:
# Generate Spark Session
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("Reading from Sockets")
    .master("local[*]")
    .getOrCreate()
)

spark

In [2]:
# Read input data

df_raw = spark.readStream.format("socket").option("host","localhost").option("port", "9999").load()

In [3]:
df_raw.printSchema()

root
 |-- value: string (nullable = true)



In [4]:
# Split the line into words
from pyspark.sql.functions import split

df_words = df_raw.withColumn("words", split(df_raw["value"], " "))

In [5]:
# Explode the list of words
from pyspark.sql.functions import explode

df_explode = df_words.withColumn("word", explode(df_words["words"])).drop("value", "words")

In [6]:
# Aggregate the words to generate count
from pyspark.sql.functions import count, lit

df_agg = df_explode.groupBy("word").agg(count(lit(1)).alias("count"))

In [ ]:
# Exec into container
# Install netcat
# Open port 9999 for connections with:
# ncat -l 9999

# Write the output to console streaming
# This also launches a spark job which you can see via the Spark UI or http://localhost:4040/jobs/
# If there are no spark jobs in the UI, the UI might be referencing the wrong kernel. Shut down ALL kernels and spin up a new one.

df_agg.writeStream.format("console").outputMode("complete").start().awaitTermination()